<a href="https://colab.research.google.com/github/cristiandarioortegayubro/BDS/blob/main/modulo.04/bds_optimizacion_005_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>


<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20Scikit-learn.png?raw=true">
</p>


 # **<font color="DeepPink">Ajuste de hiperparámetros por grid-search</font>**

<p align="justify">
♥ En el Colab anterior, vimos que los hiperparámetros pueden afectar el rendimiento de generalización de un modelo. En este Colab, mostraremos cómo optimizar los hiperparámetros utilizando un enfoque de búsqueda en cuadrícula.

 ## **<font color="DeepPink">Carga de las librerías</font>**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import plotly.express as px

 ## **<font color="DeepPink">Carga del conjunto de datos</font>**

In [ ]:
adult_census = pd.read_csv("https://raw.githubusercontent.com/cristiandarioortegayubro/BDS/main/datasets/adult_census.csv")
adult_census

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


 ## **<font color="DeepPink">Separamos la variable objetivo y las variables explicativas</font>**

<p align="justify">
👀 Asignamos a un objeto la variable objetivo:
</p>


In [ ]:
target_name = "class"
y = adult_census[target_name]
y

0         <=50K
1         <=50K
2          >50K
3          >50K
4         <=50K
          ...  
48837     <=50K
48838      >50K
48839     <=50K
48840     <=50K
48841      >50K
Name: class, Length: 48842, dtype: object

<p align="justify">
👀 Eliminamos de nuestros datos la variable objetivo y la columna <code>education-num</code> que duplica la información de la columna <code>education</code>.

In [ ]:
X = adult_census.drop(columns=[target_name, "education-num"])
X.head()

,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,?,Some-college,Never-married,?,Own-child,White,Female,0,0,30,United-States


 ## **<font color="DeepPink">Conjunto de entrenamiento y conjunto de prueba</font>**

👀 Dividimos en conjunto de entrenamiento y prueba

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

 ## **<font color="DeepPink">Pipeline</font>**

<p align="justify">
✅ Definiremos un Pipeline que va a manejar características numéricas como categóricas. El primer paso es seleccionar todas las columnas categóricas...

In [ ]:
from sklearn.compose import make_column_selector as selector

In [ ]:
categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(X)

👀 Vemos las columnas categóricas...

In [ ]:
categorical_columns

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country']

<p align="justify">
Usaremos un modelo basado en árboles de decisión (es decir, <code>HistGradientBoostingClassifier</code>). Eso significa que:

* Las variables numéricas no necesitan escala;
* Las variables categóricas se pueden tratar con un `OrdinalEncoder` incluso si el orden de codificación no es significativo;
* Para modelos basados en árboles, `OrdinalEncoder` evita tener representaciones de alta dimensión.

<br>


<p align="justify">
✅ Ahora construimos nuestro <code>OrdinalEncoder</code> pasándole las categorías conocidas.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)

In [ ]:
categorical_preprocessor

OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

<p align="justify">
✅ Luego usamos un <code>ColumnTransformer</code> para seleccionar las columnas categóricas y les aplicamos el <code>OrdinalEncoder</code>

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
preprocessor = ColumnTransformer([
    ('cat_preprocessor', categorical_preprocessor, categorical_columns)],
    remainder='passthrough', sparse_threshold=0)

In [ ]:
preprocessor

ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                  transformers=[('cat_preprocessor',
                                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

<p align="justify">
✅ Finalmente, usamos un clasificador basado en árboles (histogram gradient-boosting) para predecir si una persona gana o no más de 50 k$ al año.

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

In [ ]:
model = Pipeline([("preprocessor", preprocessor),
                  ("classifier", HistGradientBoostingClassifier(random_state=42,
                                                                max_leaf_nodes=4))])

In [ ]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                                   transformers=[('cat_preprocessor',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier',
                 HistGradientBoostingClassifier(max_leaf_nodes=4,
                                                random_state=42))])

 # **<font color="DeepPink">Ajuste mediante grid-search</font>**

<p align="justify">
Anteriormente, usamos <code>for</code> para encontrar valores óptimos de los parámetros.
<br><br>
<code>GridSearchCV</code> es una clase de <code>scikit-learn</code> que implementa una lógica muy similar con un código menos repetitivo.
Veamos cómo usar el estimador <code>GridSearchCV</code> para realizar dicha búsqueda. Dado que la búsqueda será costosa, solo exploraremos la combinación de tasa de aprendizaje y el número máximo de nodos.
<br><br>
Grid-search es una técnica utilizada para encontrar la combinación óptima de hiperparámetros para un modelo de aprendizaje automático. Los hiperparámetros son parámetros que no se aprenden directamente del conjunto de datos, pero afectan el rendimiento y la capacidad del modelo.
<br><br>
Grid-search implica especificar una cuadrícula de valores posibles para diferentes hiperparámetros y luego evaluar el rendimiento del modelo en cada combinación de valores. Esto implica entrenar y evaluar el modelo con todas las posibles combinaciones de hiperparámetros.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'classifier__learning_rate': [0.01, 0.1, 1, 10], #corchete o parentesis
    'classifier__max_leaf_nodes': (3, 10, 30)}

In [ ]:
model_grid_search = GridSearchCV(model, param_grid=param_grid, n_jobs=2, cv=2)
model_grid_search.fit(X_train, y_train)

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          sparse_threshold=0,
                                                          transformers=[('cat_preprocessor',
                                                                         OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                        unknown_value=-1),
                                                                         ['workclass',
                                                                          'education',
                                                                          'marital-status',
                                                                          'occupation',
                                                                          'relationship',
                                                                          'race',
                                                                          'sex',
                                                                          'native-country'])])),
                                       ('classifier',
                                        HistGradientBoostingClassifier(max_leaf_nodes=4,
                                                                       random_state=42))]),
             n_jobs=2,
             param_grid={'classifier__learning_rate': [0.01, 0.1, 1, 10],
                         'classifier__max_leaf_nodes': (3, 10, 30)})

<p align="justify">
✅ Finalmente, comprobamos el <code>accuracy</code> del modelo usando el conjunto de prueba.

In [ ]:
accuracy = model_grid_search.score(X_test, y_test)

In [ ]:
print("")
print(f"The test accuracy score of the grid-searched pipeline is: "f"{accuracy:.2f}")


The test accuracy score of the grid-searched pipeline is: 0.88


<p align="justify">
🛑 Advertencia
<br><br>
Tenga en cuenta que la evaluación normalmente debe realizarse a través de la validación cruzada al proporcionar <code>model_grid_search</code> como modelo para la función <code>cross_validate</code>.
<br><br>
Aquí, usamos una sola división de entrenamiento y prueba para evaluar <code>model_grid_search</code>. En un Colab futuro, entraremos en más detalles sobre la validación cruzada anidada, cuando se use la validación cruzada tanto para el ajuste de hiperparámetros, como para la evaluación del modelo.

<p align="justify">
El estimador <code>GridSearchCV</code> toma un parámetro <code>param_grid</code> que define todos los hiperparámetros y sus valores asociados. El <code>grid-search</code> se encargará de crear todas las combinaciones posibles y probarlas.
<br><br>
El número de combinaciones será igual al producto del número de valores a explorar para cada parámetro (por ejemplo, en nuestro ejemplo, combinaciones $4 × 3$). Por lo tanto, agregar nuevos parámetros con sus valores asociados para ser explorados se vuelve rápidamente computacionalmente costoso.
<br><br>
Una vez que se ajusta grid-search, se puede usar como cualquier otro predictor llamando a <code>predict</code> y <code>predict_proba</code>. Internamente, utilizará el modelo con los mejores parámetros encontrados durante el <code>fit</code>.
<br><br>
Ahora vamos a obtener las predicciones para las $5$ primeras muestras usando el estimador con los mejores parámetros.

In [ ]:
model_grid_search.predict(X_test.iloc[0:5])

array([' <=50K', ' <=50K', ' >50K', ' <=50K', ' >50K'], dtype=object)

<p align="justify">
👀 Puede conocer estos parámetros mirando el atributo <code>best_params_</code>.

In [ ]:
print("")
print(f"The best set of parameters is: "
      f"{model_grid_search.best_params_}")


The best set of parameters is: {'classifier__learning_rate': 0.1, 'classifier__max_leaf_nodes': 30}


<p align="justify">
La <code>accuracy</code> y los mejores parámetros del Pipeline buscados  (grid-searched pipeline) son similares a los que encontramos anteriormente, donde buscamos los mejores parámetros "a mano" a través de un bucle <code>for</code>.
<br><br>
Además, podemos inspeccionar todos los resultados que se almacenan en el atributo <code>cv_results_</code> de la búsqueda en grid-search. Filtraremos algunas columnas específicas de estos resultados.

In [ ]:
cv_results = pd.DataFrame(model_grid_search.cv_results_).sort_values("mean_test_score", ascending=False)
cv_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__learning_rate,param_classifier__max_leaf_nodes,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
5,0.910507,0.081211,0.337949,0.024030,0.1,30,"{'classifier__learning_rate': 0.1, 'classifier...",0.868912,0.867213,0.868063,0.000850,1
4,0.681341,0.003319,0.321939,0.005325,0.1,10,"{'classifier__learning_rate': 0.1, 'classifier...",0.866783,0.866066,0.866425,0.000359,2
7,0.231593,0.014162,0.151712,0.005427,1,10,"{'classifier__learning_rate': 1, 'classifier__...",0.858648,0.862408,0.860528,0.001880,3
6,0.258250,0.008097,0.166133,0.002430,1,3,"{'classifier__learning_rate': 1, 'classifier__...",0.859358,0.859514,0.859436,0.000078,4
8,0.246942,0.012513,0.156900,0.014335,1,30,"{'classifier__learning_rate': 1, 'classifier__...",0.855536,0.856129,0.855832,0.000296,5


<p align="justify">
Centrémonos en las columnas más interesantes y acortemos los nombres de los parámetros para eliminar el prefijo <code>param_classifier__</code> para facilitar la lectura:

In [ ]:
column_results = [f"param_{name}" for name in param_grid.keys()]

In [ ]:
column_results += ["mean_test_score", "std_test_score", "rank_test_score"]

In [ ]:
cv_results = cv_results[column_results]

In [ ]:
cv_results

,param_classifier__learning_rate,param_classifier__max_leaf_nodes,mean_test_score,std_test_score,rank_test_score
5,0.1,30,0.868063,0.000850,1
4,0.1,10,0.866425,0.000359,2
7,1,10,0.860528,0.001880,3
6,1,3,0.859436,0.000078,4
8,1,30,0.855832,0.000296,5
3,0.1,3,0.853266,0.000515,6
2,0.01,30,0.843330,0.002917,7
1,0.01,10,0.817832,0.001124,8
0,0.01,3,0.797166,0.000715,9
11,10,30,0.288200,0.050539,10


In [ ]:
def shorten_param(param_name):
    if "__" in param_name:
        return param_name.rsplit("__", 1)[1]
    return param_name

In [ ]:
cv_results = cv_results.rename(shorten_param, axis=1)
cv_results

,learning_rate,max_leaf_nodes,mean_test_score,std_test_score,rank_test_score
5,0.1,30,0.868063,0.000850,1
4,0.1,10,0.866425,0.000359,2
7,1,10,0.860528,0.001880,3
6,1,3,0.859436,0.000078,4
8,1,30,0.855832,0.000296,5
3,0.1,3,0.853266,0.000515,6
2,0.01,30,0.843330,0.002917,7
1,0.01,10,0.817832,0.001124,8
0,0.01,3,0.797166,0.000715,9
11,10,30,0.288200,0.050539,10


<p align="justify">
Con solo $2$ parámetros, es posible que queramos visualizar grid-search como un mapa de calor. Necesitamos transformar nuestro <code>cv_results</code> en un <code>DataFrame</code> donde:

- Las filas corresponderán a los valores de la tasa de aprendizaje;
- Las columnas corresponderán al número máximo de hojas;
- El contenido del <code>DataFrame</code> serán las puntuaciones medias de las pruebas.

In [ ]:
pivoted_cv_results = cv_results.pivot_table(
    values="mean_test_score", index=["learning_rate"],
    columns=["max_leaf_nodes"])

In [ ]:
pivoted_cv_results

max_leaf_nodes,3,10,30
learning_rate,,,
0.01,0.797166,0.817832,0.843330
0.10,0.853266,0.866425,0.868063
1.00,0.859436,0.860528,0.855832
10.00,0.283476,0.262564,0.288200


👀 Generando el mapa de calor

In [ ]:
pivoted_cv_results.columns = ["3","10","30"]
pivoted_cv_results.index = ["0.01", "0.10", "1.00", "10.00"]

In [ ]:
fig = px.imshow(pivoted_cv_results.round(2),
                text_auto=True,
                aspect="auto",
                title="Mapa de Calor",
                labels=dict(x="Max leaf nodes",
                            y="Learning rate",
                            color="Mean Test Score"))

fig.show()

<p align="justify">
📊 El mapa de calor destaca lo siguiente:

- Para valores demasiado altos de `learning_rate`, el rendimiento de generalización del modelo está degradado y no se puede corregir ajustando con  `max_leaf_nodes`.
- Se observa que la elección óptima de `max_leaf_nodes` depende del valor de `learning_rate`.
- En particular, se observa una "diagonal" de buenos modelos con una `accuracy` cerca de 0.87: cuando el valor de `max_leaf_nodes` es  aumentado, se debe disminuir el valor de `learning_rate`, para conservar una buena precisión.

<br>
<p align="justify">
🛑 El significado preciso de esos dos parámetros se explicará más adelante.
<br><br>
Por ahora notaremos que, en general, <b>no existe un único parámetro óptimo</b>, 4 modelos de las 12 configuraciones de parámetros alcanzan la máxima
precisión.

 # **<font color="DeepPink">Conclusiones</font>**

<p align="justify">
👀 En este colab nosotros:
<br><br>
✅ Cargamos los datos de un archivo <code>CSV</code> usando <code>Pandas</code>.<br>
✅ Generamos un Pipeline.
<br>  
✅ Separamos en un conjunto de entrenamiento y un conjunto de prueba.
<br>
✅ Optimizamos los hiperparámetros de un modelo predictivo a través del grid-search.
<br>
✅ Constatamos que buscar más de dos hiperparámetros es demasiado costoso.
<br>
✅ Vimos que una búsqueda en grid-search no necesariamente encuentra una solución óptima.



<br>
<br>
<p align="center"><b>
💗
<font color="DeepPink">
Hemos llegado al final de nuestro colab, a seguir codeando...
</font>
</p>
<br>
<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>

---
